In [113]:
import pandas as pd
import datetime
import numpy as np

In [114]:
data = pd.read_csv("barro_data.csv", index_col=['code', 'year'])
country_list = pd.read_csv("../data_source/country_code_list.csv", index_col='country')
data.head()

gini_DS quality  GDP_PWT  GDP_PC_PWT  gov_consumption_WB  \
code year                                                             
AAB  1981      NaN     NaN      NaN         NaN                 NaN   
     1982      NaN     NaN      NaN         NaN                 NaN   
     1983      NaN     NaN      NaN         NaN                 NaN   
     1984      NaN     NaN      NaN         NaN                 NaN   
     1985      NaN     NaN      NaN         NaN                 NaN   

           years_schooling  democracy_index  GDP_MP_WB  GDP_growth_WB  \
code year                                                               
AAB  1981              NaN            12.48        NaN            NaN   
     1982              NaN            12.48        NaN            NaN   
     1983              NaN            12.48        NaN            NaN   
     1984              NaN             7.91        NaN            NaN   
     1985              NaN             7.91        NaN            NaN   

           inflation_WB  fertility_WB  
code year                              
AAB  1981           NaN           NaN  
     1982           NaN           NaN  
     1983           NaN           NaN  
     1984           NaN           NaN  
     1985           NaN           NaN

We only data keep posterior to 1960. We also remove the countries that do not appear in the list of countries. 

In [115]:
data = data.query('year >= 1955')
data = data.query("code in " + str(country_list['code'].values.tolist()))
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 13400 entries, (ABW, 1960.0) to (ZWE, 2014.0)
Data columns (total 11 columns):
gini_DS               2321 non-null float64
quality               2322 non-null object
GDP_PWT               9336 non-null float64
GDP_PC_PWT            9336 non-null float64
gov_consumption_WB    8897 non-null float64
years_schooling       2099 non-null float64
democracy_index       3880 non-null float64
GDP_MP_WB             9634 non-null float64
GDP_growth_WB         9607 non-null float64
inflation_WB          7748 non-null float64
fertility_WB          11733 non-null float64
dtypes: float64(10), object(1)
memory usage: 1.2 MB


# Bad quality data

In his study, Barro change 48 observations quality to 'accept' (cf. note 11), these observations have been gathered in the file added_observations.csv.

In [116]:
added_obs = pd.read_csv("added_observations.csv", names=['country', 'year'])
for row in added_obs.iterrows():
    for year in (np.array([-5, -4, -3, -2, -1, 1, 2, 3, 4, 5]) + row[1]['year']):
        added_obs.loc[len(added_obs)] = [row[1]['country'], year]
added_obs['code'] = added_obs['country'].map(lambda x: country_list.loc[x]['code'])
len(data[data['quality']=='accept'])

609

In [117]:
for obs in added_obs.iterrows():
    try:
        data.loc[(obs[1]['code'], obs[1]['year']), 'quality'] = 'accept'
    except KeyError:
        None
len(data[data['quality']=='accept'])

1229

In [118]:
data = data[data['quality'].isnull() | data['quality'].map(lambda x: x=='accept')]

# Resampling the data

In [119]:
period = "5AS"
re_data = pd.DataFrame()
for country in set(data.index.levels[0].values):
    sel = data.loc[country]
    sel.index = sel.index.map(lambda x: datetime.datetime(int(x), 1, 1, 0, 0, 0))
    sel = sel.resample(period)
    sel['code'] = [country] * len(sel)
    re_data = pd.concat([re_data, sel], axis=0)
re_data = re_data.sort_index()

In [120]:
re_data['year'] = re_data.index
re_data = re_data.set_index(['code', 'year'])
re_data = re_data.sort_index()
re_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2418 entries, (ABW, 1960-01-01 00:00:00) to (ZWE, 2010-01-01 00:00:00)
Data columns (total 11 columns):
GDP_MP_WB             1686 non-null float64
GDP_PC_PWT            1685 non-null float64
GDP_PWT               1685 non-null float64
GDP_growth_WB         1724 non-null float64
democracy_index       682 non-null float64
fertility_WB          2136 non-null float64
gini_DS               392 non-null float64
gov_consumption_WB    1565 non-null float64
inflation_WB          1400 non-null float64
quality               0 non-null object
years_schooling       1644 non-null float64
dtypes: float64(10), object(1)
memory usage: 226.7 KB


# Ploting the data

In [125]:
re_data[~re_data['gini_DS'].isnull()].info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 392 entries, (ARG, 1955-01-01 00:00:00) to (ZWE, 1990-01-01 00:00:00)
Data columns (total 11 columns):
GDP_MP_WB             340 non-null float64
GDP_PC_PWT            373 non-null float64
GDP_PWT               373 non-null float64
GDP_growth_WB         341 non-null float64
democracy_index       187 non-null float64
fertility_WB          367 non-null float64
gini_DS               392 non-null float64
gov_consumption_WB    334 non-null float64
inflation_WB          295 non-null float64
quality               0 non-null object
years_schooling       339 non-null float64
dtypes: float64(10), object(1)
memory usage: 36.8 KB
